# Exploring Mental Health Discourse on Reddit

### Import Libraries 

In [1]:
import os
import datetime
import time
import re
import requests
import random

import pandas as pd
import praw

from nltk.corpus import stopwords

import string 
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

In [2]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Connecting to Reddit API

*The code was modified based on resources available at https://github.com/rogerfitz/tutorials/blob/master/subreddit_analysis/0_Setup.ipynb and a tutorial accessed via https://www.youtube.com/watch?v=a3zQdlLwEb4.*

In [3]:
# establish connection with reddit API
reddit = praw.Reddit(
    client_id="81TBuwGeFdL5ZfijNpLxNw",
    client_secret="fn2ztcn8ojr3vewO93h46nZ2YY47mw",
    user_agent="Academic_Winner_5303",
)

#### Obtaining Posts from r/mentalhealth

In [4]:
# specify the subreddit
subreddit_mh = 'MentalHealth'

# fetch posts fromsubreddit
subreddit = reddit.subreddit(subreddit_mh)
word_to_check = "mental health"

In [5]:
for submission in reddit.subreddit('mentalhealth').hot(limit=None):
    print(submission.title)

Men's Mental Health Month
Warning regarding DM's and chatgroups offering "help".
What’s the point of life
What are your tips for adhd at work ? 
Feeling like shit when on my period 
How did your parents' actions leave you traumatized?
What is your song? 
People with anxiety and depression, how did each condition respond to medicine or therapy?
I have a friend who's bipolar
Hereditary Lack of Empathy 
Can you be honest with depressed individual and tell them how their actions affected you even if it may make them feel awful?
I think about this 
On the edge 
I've been having some really down days.
I don't think I'm getting better
1st steps to addressing mental health???
can some trauma cause huge memory problems or is it just the way i am
has medication worked for you?
Will it get better ? 
I cant deal with this stuff anymore
i havent been genuinely happy since i got bullied
My life is a hell of voices and unending emptiness and dread
Can I become less emotionally numb and more empatheti

Do I have smiling depression?
Women around my age make me uncomfortable 
Jury duty tomorrow is causing me stress
I think I'm just going to end it all.
I feel like a person with good mental health who’s pretending to be a in a worse state then I am.
How do I not feel nothing
My parents are attentive towards my younger sister’s mental health but they ignore mine
Does this happen to you? How do you navigate thru this? 
Someone with depression and who I thought was a friend ditched me multiple times and would vent and talk at me but never hang out. Is that normal to be only used as emotional support?
Is it addiction?
what is wrong with me
I don’t know what I’m doing
Insights from Gabor Maté
Therapist. But why? 
Thank you ❤️
Someone with depression and whoI thought was a friend ditched me multiple times and would vent and talk at me but never hang out. Is that normal to be only used as emotional support?
Already setting up to get a therapist, but what do you guys think is wrong with me?
Med

How do i stop caring about others opnion
I don’t know how to properly cope with my mood swings
Its very hard to work with this....
I just came home from the pyschiatric ward and don't know how to cope 
I push people away and I need to know how to stop running away from being close to people
Is this harassment?
Bad break up 
How is anyone happy in this world.
I need help
Job search
I turned 25 today, what should I do to not waste the next 5 years
23F i feel so hopeless and lonely guys..idk how do to go on living everyday
Person changed bio to something weird
How do you just keep moving forward
Unsure of what this is
Why do i find media stressful or unintresting?
Why are schools lying to us?
Insomnia and Venlafaxine???
How would you react to this?
I'm not coping 
i have no sense of identity. i have no idea who i am anymore, nor have i ever.
My only “friend” is AI 
I don't know what to do
I continue to suffer from my self loathing and I don’t know what else to do.
Imaginary friends 
Vent,

I have a problem
i met my favorite athlete, not happy abt it.
Does this happen to anyone else?
Please help, I want to end it all.
How do I make time for myself while grieving?
I suffer from visual intrusions
Im done, i dont think i can recover.
Visions of terrible things
He left after 4 years for a girl he knows for a week.
Called out of work today 
I can’t tell if I’m still clinically depressed or if this is just a matter of outlook
I always feel weird 
Avid computer user developing Cyberphobia
constant fear of being tortured
I want to talk
Dealing with panic attacks 
How do you work when you dont even want to live
is this people pleasing behavior 
Struggling to keep food down, seeing psych tomorrow
Ive been having nihilistic/pessimistic thoughts
Life with ADHD 
I don't feel good and I think I have a real problem. 
i feel physically uncomfortable at all times
i need help please this is getting really fucking bad
Need support/help - depressive episode 
Am I dealing with PTSD?
Number fo

I have a habit of ignoring my gut feelings, but is it really an intuition?
inpatient therapy for the first time 
Please help me out
If my childhood wasn’t that bad, how did I end up like this? 
Genuinely can't find any psychiatric services under medicaid
random wave of depression and hopelessness 
Dad being doubtful 
Why am I obsessing over this? 
how do i handle my sister ? shes too controlling and my family is too innocent and not strict, what should i do as a brother?
Autistic Schizoid? Help me please 🥹
If you join the military, you are destined to have some issues. (I am venting and also trying to reach a Veteran audience).
I'm so lonely i talk to myself like there actually is a another person with me..
I don’t even know what medication to try next…
I just really miss her 
Returned to my old depression habits
Mental Health is bad right nka
I need help with my Imposter Syndrome 
I feel like Im going insane
why is mental health suddenly unimportant when the person suffering is aware 

In [6]:
subreddit=reddit.subreddit("mentalhealth")
subreddit.over18

False

In [7]:
subreddit.subscribers

468578

In [8]:
subreddit.__dict__

{'display_name': 'mentalhealth',
 '_reddit': <praw.reddit.Reddit at 0x7fbdca33bb80>,
 '_fetched': True,
 '_path': 'r/mentalhealth/',
 'user_flair_background_color': None,
 'submit_text_html': '<!-- SC_OFF --><div class="md"><p>All promotional, survey, research study, or business posts require moderator approval. Please message a moderator to check if your post is suitable. Avoid posting direct links without some context.</p>\n</div><!-- SC_ON -->',
 'restrict_posting': True,
 'user_is_banned': None,
 'free_form_reports': True,
 'wiki_enabled': True,
 'user_is_muted': None,
 'user_can_flair_in_sr': None,
 'header_img': None,
 'title': 'Mental Health',
 'allow_galleries': False,
 'icon_size': None,
 'primary_color': '#005ba1',
 'active_user_count': 50,
 'icon_img': '',
 'display_name_prefixed': 'r/mentalhealth',
 'accounts_active': 50,
 'public_traffic': False,
 'subscribers': 468578,
 'user_flair_richtext': [],
 'name': 't5_2qirg',
 'quarantine': False,
 'hide_ads': False,
 'prediction_

In [9]:
# define function to fetch posts from subreddit ('MentalHealthSupport')
def fetch_posts(subreddit, category='hot', limit=None):
    if category == 'top':
        posts = subreddit.top(time_filter='year', limit=limit)
    elif category == 'rising':
        posts = subreddit.rising(limit=limit)
    elif category == 'new':
        posts = subreddit.new(limit=limit)
    else:
        posts = subreddit.hot(limit=limit)
    
    rows = []
    for submission in posts:
        row = {
            'title': submission.title,
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': submission.created_utc,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit,
            'author': str(submission.author),
            'media_only': submission.media_only,
            'permalink': submission.permalink,
            'num_comments': submission.num_comments
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [10]:
mental_health_top = fetch_posts(subreddit, category='top')
mental_health_hot = fetch_posts(subreddit, category='hot')
mental_health_rising = fetch_posts(subreddit, category='rising')
mental_health_new = fetch_posts(subreddit, category='new')

In [11]:
# concatenate dataframes
mental_health_df = pd.concat([mental_health_top,
                               mental_health_hot, 
                               mental_health_rising, 
                               mental_health_new],
                              ignore_index=True)

# drop duplicates
mental_health_df = mental_health_df.drop_duplicates()

In [12]:
mental_health_df['upvotes']=mental_health_df['score']/mental_health_df['upvote_ratio']
mental_health_df['downvotes']=mental_health_df['upvotes']-mental_health_df['score']
mental_health_df['upvotes/subscribers']=mental_health_df['upvotes']/subreddit.subscribers

# convert created_utc to datetimr format 
mental_health_df['created_utc'] = pd.to_datetime(mental_health_df['created_utc'], unit='s')

mental_health_df.sample(5)

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
1146,Do I have smiling depression?,1,1.00,2024-06-13 06:20:20,"I can laugh, I can smile, I can not think abou...",mentalhealth,Ok-Cheek7283,False,/r/mentalhealth/comments/1des79u/do_i_have_smi...,1,1.000000,0.000000,0.000002
715,22 male and i’m so fucking lost in life I don’...,47,0.95,2024-03-23 14:55:08,I live in asia and whilst growing up during my...,mentalhealth,AdministrativeAsk598,False,/r/mentalhealth/comments/1blu33f/22_male_and_i...,15,49.473684,2.473684,0.000106
663,people who are only child with a broken family...,53,0.96,2023-09-13 12:23:59,I've been realizing that i'm on my own now. Th...,mentalhealth,rekkenn,False,/r/mentalhealth/comments/16hlmtv/people_who_ar...,32,55.208333,2.208333,0.000118
1415,How am I supposed to accept that I have missed...,1,1.00,2024-06-12 01:56:11,For 25 years of my existence I have lived in a...,mentalhealth,biiiiigdong,False,/r/mentalhealth/comments/1ddvc5n/how_am_i_supp...,0,1.000000,0.000000,0.000002
1615,Acceptance stage of Depression,1,1.00,2024-06-11 05:13:35,"Hey Guys, \nYesterday I admitted to myself tha...",mentalhealth,genuineskuxx,False,/r/mentalhealth/comments/1dd6d3l/acceptance_st...,0,1.000000,0.000000,0.000002


In [13]:
mental_health_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2043 entries, 0 to 2900
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   title                2043 non-null   object        
 1   score                2043 non-null   int64         
 2   upvote_ratio         2043 non-null   float64       
 3   created_utc          2043 non-null   datetime64[ns]
 4   selftext             2043 non-null   object        
 5   subreddit            2043 non-null   object        
 6   author               2043 non-null   object        
 7   media_only           2043 non-null   bool          
 8   permalink            2043 non-null   object        
 9   num_comments         2043 non-null   int64         
 10  upvotes              2043 non-null   float64       
 11  downvotes            2043 non-null   float64       
 12  upvotes/subscribers  2043 non-null   float64       
dtypes: bool(1), datetime64[ns](1), fl

In [14]:
# create directory to store data 
os.makedirs('data', exist_ok=True)

# create csv 
mental_health_df.to_csv('data/mental_health_posts.csv', index=False)

#### Obtaining Posts  from r/MentalHealthSupport 

In [15]:
# specify the subreddit
subreddit_mhs = 'MentalHealthSupport'

# fetch posts fromsubreddit
subreddit = reddit.subreddit(subreddit_mhs)
word_to_check = "mental health"

In [16]:
for submission in reddit.subreddit('mentalhealthsupport').hot(limit=None):
    print(submission.title)

Am I a loser? 
Feeling lonely, depressed, and with intrusive thoughts about the past
How to overcome work anxiety?
Help me please
At a loss
Grieving, parent and grandparent - just want to talk
Something's seriously wrong with me...
Not even sure wtf to title it but I’m not feeling good overall 
How to deal with people like this
How to help someone grief and come to terms with their reality? 
I need help with my friend. 
Falling into an episode again
For people who have lost years of their life to mental health issues, how did you start getting your life back?
Somewhere and someone to vent to 
I can’t do this
Another therapist has given up on me. What do I do now?
Pls help why do I keep seeing things
Over thinking?
Severely struggling with relationship health and control issues. 
Help 
I Do Everything For My Mental Health, But Still Depressed - Why?
I don't feel real.
Help with understanding what is causing the angry behavior?
Struggling to get up in the morning, tired all the time
I ca

I feel like I’m losing the battle 
My mental illness is ruining my life
Fear of nothing
Need advice
Work and mental health - do I get out before it gets worse? 
Just a little vent
Gender discriminations in health institutions
I need awnsers
Just abit curious
I feel like I have BPD but I’m a minor and don’t know where to get it checked
weird things going on, seekin for help/support
I need help with my mental health. Maybe it’s depression 
Just a vent
Partner self harms and I don't know what to do 
Lost in my journey
I hate everybody
My friend is suicidal and I need help
How anxiety impacts my life 
Body image help
In the edge
Am I faking of having a mental illness?
Anxiety and depression when starting a new job 
I so close to doing it 
I did what I was supposed to
I need help, but at what cost?
Can't walk to school
Honestly 
My manager is affecting my mental health
Not sure what to do here, any advice?
How do I react, I can't even see her in face anymore.
reason to live?
Just want some 

just tired yk
What do I do?
I am pregnant and need fact based advice about a medication..
Eating issues and depression after experiencing sa
Delayed grief-childhood trauma
How do I get over how toxic I used to be? 
Do i have a personality disorder?
TW: Was this an attempt?
I was doing so well for a while, then I read what “Quiet on Set” is all about
What can I do with my derealization?
Do you ever get this feeling?
I am worried that my partner feels uncomfortable and maybe unsafe in my presence when I am experiencing psychotic/schizophrenic symptoms or episodes. 
I do want mango, tbh.
How do I approach my friends making fun of my learning difficulties in a not so nice way?
Anybody else struggle with getting overly attached?
ex-bf is leaving the country forever and I'm feeling bad about it 
My bf left me 
When is it enough to go to therapy?
everything is changing and it’s hard to come to terms with it.
Mental Health and STI (trigger warning) 
Time is fast forwarding
(TW: suicide) My fri

Am I being soft?
How to I tell people I'm ok 
My father died a year ago, and I never shed a tear. Help. 
Still feeling guilty years later
Let my self down. Lost 
What to do when your mother is suicidal? 
I chose “success story” but this is still a constant battle, its just one of many past and future success stories  
How to cope with pressure.
Mental Health Help
I got drunk in a bar, was speaking very loudly. Called a girl and talked dirty with her very loudly. People around me were laughing and judging me. Some came and told me to behave myself. I was abusing people as well. 
Need help at oh-dark-thirty
Will I feel better soon?
what does it mean if i pick fights with most of my peers
Any good tips on stopping trichotillomania?
Struggling with mental health, finishing school, and wanting to live
Mental health advice
Would my suicide be selfish? 
Spiraling hardcore
Relationship Break
28F looking for insight, can't afford therapy or psychiastrist
If my message is delivered
i really need

In [17]:
subreddit=reddit.subreddit('mentalhealthsupport')
subreddit.over18

False

In [18]:
subreddit.subscribers

46156

In [19]:
subreddit.__dict__

{'display_name': 'MentalHealthSupport',
 '_reddit': <praw.reddit.Reddit at 0x7fbdca33bb80>,
 '_fetched': True,
 '_path': 'r/MentalHealthSupport/',
 'user_flair_background_color': None,
 'submit_text_html': '<!-- SC_OFF --><div class="md"><p>Please provide context to links since they require moderator approval. Promotional posts are not permitted.  Message the moderators if you have any query. Make sure your post follows the rules.</p>\n\n<p>This subreddit is optimized for the new Reddit redesign and official app. Please use the newer design to enjoy the subreddit to its fullest, thank you!</p>\n</div><!-- SC_ON -->',
 'restrict_posting': True,
 'user_is_banned': None,
 'free_form_reports': True,
 'wiki_enabled': True,
 'user_is_muted': None,
 'user_can_flair_in_sr': None,
 'header_img': None,
 'title': 'Mental Health Support & Azal',
 'allow_galleries': False,
 'icon_size': None,
 'primary_color': '#2a4fd5',
 'active_user_count': 12,
 'icon_img': '',
 'display_name_prefixed': 'r/Mental

In [20]:
# define function to fetch posts from subreddit ('MentalHealthSupport')
def fetch_posts(subreddit, category='hot', limit=None):
    if category == 'top':
        posts = subreddit.top(time_filter='year', limit=limit)
    elif category == 'rising':
        posts = subreddit.rising(limit=limit)
    elif category == 'new':
        posts = subreddit.new(limit=limit)
    else:
        posts = subreddit.hot(limit=limit)
    
    rows = []
    for submission in posts:
        row = {
            'title': submission.title,
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': submission.created_utc,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit,
            'author': str(submission.author),
            'media_only': submission.media_only,
            'permalink': submission.permalink,
            'num_comments': submission.num_comments
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [21]:
mental_health_support_top = fetch_posts(subreddit, category='top')
mental_health_support_hot = fetch_posts(subreddit, category='hot')
mental_health_support_rising = fetch_posts(subreddit, category='rising')
mental_health_support_new = fetch_posts(subreddit, category='new')

In [22]:
# concatenate dataframes
mental_health_support_df = pd.concat([mental_health_support_top, 
                                      mental_health_support_hot, 
                                      mental_health_support_rising, 
                                      mental_health_support_new], 
                                     ignore_index=True)

# drop duplicates
mental_health_support_df = mental_health_support_df.drop_duplicates()

In [23]:
mental_health_support_df.head()

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments
0,I feel hopeless.,157,0.99,1.709293e+09,"Hi, I’m 24F. I stay home 24/7 , everyday. I do...",MentalHealthSupport,Odd-View-667,False,/r/MentalHealthSupport/comments/1b3s6ad/i_feel...,154
1,What changes your negative thoughts/ has made ...,45,1.00,1.709444e+09,I am learning that affirmations and the like c...,MentalHealthSupport,Bananasloog1998,False,/r/MentalHealthSupport/comments/1b59lzz/what_c...,21
2,what is the point of living?,43,0.92,1.709354e+09,\ncontent warning for suicide.\n\n\n\nI dont u...,MentalHealthSupport,penjamincranklin,False,/r/MentalHealthSupport/comments/1b4fkko/what_i...,24
3,I need a hug,23,0.92,1.709557e+09,I wanna end me.,MentalHealthSupport,Positive-Parthlow,False,/r/MentalHealthSupport/comments/1b6a8vz/i_need...,26
4,What are you like after a hug,22,1.00,1.709738e+09,If I hug someone I just try my best not to cry...,MentalHealthSupport,itsRileyigitbanned,False,/r/MentalHealthSupport/comments/1b8299d/what_a...,17


In [24]:
# calculate upvotes and downvotes
mental_health_support_df['upvotes']=mental_health_support_df['score']/mental_health_support_df['upvote_ratio']
mental_health_support_df['downvotes']=mental_health_support_df['upvotes']-mental_health_support_df['score']
mental_health_support_df['upvotes/subscribers']=mental_health_support_df['upvotes']/subreddit.subscribers

# convert 'created_utc' to proper format 
mental_health_support_df['created_utc'] = pd.to_datetime(mental_health_support_df['created_utc'], unit='s')

mental_health_support_df.sample(5)

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
1481,Mental Health,1,1.0,2024-05-23 14:20:01,Has anyone ever had an uneasy feeling leaving ...,MentalHealthSupport,Important-Zucchini41,False,/r/MentalHealthSupport/comments/1cytxog/mental...,0,1.0,0.0,0.000022
327,"People with agoraphobia, how do you deal/cope ...",3,1.0,2024-03-26 14:08:51,I've posted before that agoraphobia is ruining...,MentalHealthSupport,DepressedJellyBean14,False,/r/MentalHealthSupport/comments/1bo8t9o/people...,1,3.0,0.0,0.000065
211,Had a really shitty day and just need someone ...,4,1.0,2024-03-21 04:32:22,I had a great job that I loved. I was horribly...,MentalHealthSupport,WitchingHour14,False,/r/MentalHealthSupport/comments/1bjy8h2/had_a_...,7,4.0,0.0,0.000087
1126,am I a bad person?,1,1.0,2024-06-04 20:14:41,Im not sure what started this but it’s startin...,MentalHealthSupport,satcnic,False,/r/MentalHealthSupport/comments/1d86wwb/am_i_a...,0,1.0,0.0,0.000022
674,Extended Period of Severe Depressive Symptoms,2,1.0,2024-03-11 02:02:27,I don’t want to get into too many details- but...,MentalHealthSupport,TomatoNumerous4766,False,/r/MentalHealthSupport/comments/1bbrtb0/extend...,1,2.0,0.0,0.000043


In [25]:
# create directory 
output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)

# create csv 
mental_health_support_df.to_csv('data/mental_health_support_posts.csv', index=False)

In [26]:
mental_health_support_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1927 entries, 0 to 2848
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   title                1927 non-null   object        
 1   score                1927 non-null   int64         
 2   upvote_ratio         1927 non-null   float64       
 3   created_utc          1927 non-null   datetime64[ns]
 4   selftext             1927 non-null   object        
 5   subreddit            1927 non-null   object        
 6   author               1927 non-null   object        
 7   media_only           1927 non-null   bool          
 8   permalink            1927 non-null   object        
 9   num_comments         1927 non-null   int64         
 10  upvotes              1927 non-null   float64       
 11  downvotes            1927 non-null   float64       
 12  upvotes/subscribers  1927 non-null   float64       
dtypes: bool(1), datetime64[ns](1), fl

#### Combine Dataframes (r/MentalHealth and  r/MentalHealthSupport)

In [27]:
# combine both df into one
df = pd.concat([mental_health_support_df, 
                mental_health_df],
               ignore_index=True)

In [28]:
# create csv 
df.to_csv('data/reddit_posts.csv', index=False)

In [29]:
df.info()

df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3970 entries, 0 to 3969
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   title                3970 non-null   object        
 1   score                3970 non-null   int64         
 2   upvote_ratio         3970 non-null   float64       
 3   created_utc          3970 non-null   datetime64[ns]
 4   selftext             3970 non-null   object        
 5   subreddit            3970 non-null   object        
 6   author               3970 non-null   object        
 7   media_only           3970 non-null   bool          
 8   permalink            3970 non-null   object        
 9   num_comments         3970 non-null   int64         
 10  upvotes              3970 non-null   float64       
 11  downvotes            3970 non-null   float64       
 12  upvotes/subscribers  3970 non-null   float64       
dtypes: bool(1), datetime64[ns](1), fl

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
2511,Just done. . .,61,1.00,2023-10-23 21:25:47,I have tried everything and nothing helps. \nI...,mentalhealth,No-Understanding2276,False,/r/mentalhealth/comments/17evm4o/just_done/,59,61.000000,0.000000,0.000130
457,Venting,3,1.00,2024-03-04 23:09:46,I live with my abusive mother In extreme pover...,MentalHealthSupport,VeterinarianOk1540,False,/r/MentalHealthSupport/comments/1b6pb5m/venting/,0,3.000000,0.000000,0.000065
2222,I despise men and lose sleep after multiple vi...,116,0.78,2024-01-02 14:38:55,I am 22F. The hatred and anger I feel about me...,mentalhealth,NDHrealty,False,/r/mentalhealth/comments/18wqwfc/i_despise_men...,99,148.717949,32.717949,0.000317
3101,Test day panic attack,2,1.00,2024-06-12 23:39:20,"Hi all, I’m (42M) ADHD and have started an ext...",mentalhealth,winedood,False,/r/mentalhealth/comments/1del11s/test_day_pani...,0,2.000000,0.000000,0.000004
2038,What mental illness do you think jesus would h...,205,0.82,2023-10-23 23:37:05,A lot of spiritual experiences that people hav...,mentalhealth,Optimal-Yesterday952,False,/r/mentalhealth/comments/17eymgx/what_mental_i...,117,250.000000,45.000000,0.000534


Tokenization/normalization

In [30]:
# Tokenization and Normalization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [31]:
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    # Remove stop words and punctuation
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [32]:
# Apply preprocessing to the 'selftext' column
df['tokens'] = df['selftext'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else [])

# Descriptive stats
df['token_count'] = df['tokens'].apply(len)
numerical_stats = df.describe()
token_count_stats = df['token_count'].describe()

# Most common words
all_tokens = [token for tokens in df['tokens'] for token in tokens]
fdist = FreqDist(all_tokens)
most_common_words = fdist.most_common(10)

# print stats
print("Numerical Statistics:\n", numerical_stats)
print("\nToken Count Statistics:\n", token_count_stats)
print("\nMost Common Words:\n", most_common_words)

Numerical Statistics:
              score  upvote_ratio  num_comments      upvotes    downvotes  \
count  3970.000000   3970.000000   3970.000000  3970.000000  3970.000000   
mean     29.455164      0.961637     16.952393    30.922598     1.467434   
std      70.212266      0.087824     39.486347    73.250370     4.741749   
min       0.000000      0.110000      0.000000     0.000000     0.000000   
25%       1.000000      0.970000      0.000000     1.000000     0.000000   
50%       2.000000      1.000000      1.000000     2.000000     0.000000   
75%      28.750000      1.000000     13.000000    29.336735     0.813953   
max     930.000000      1.000000    520.000000   948.979592    60.923077   

       upvotes/subscribers  token_count  
count          3970.000000  3970.000000  
mean              0.000094    95.656927  
std               0.000165    97.215186  
min               0.000000     0.000000  
25%               0.000014    36.000000  
50%               0.000043    69.000000 